# Pull

In [1]:
import random
from gaspy.gasdb import get_adsorption_docs


extra_projections = {'atoms': '$atoms',
                     'results': '$results',
                     'calc': '$calc'}
docs = get_adsorption_docs(extra_projections=extra_projections)
random.shuffle(docs)

/home/GASpy/gaspy/defaults.py:293: UserWarning: You are using adsorption document filters for an adsorbate (None) that we have not yet established valid energy bounds for. We are accepting anything in therange between -50 and 50 eV.
  UserWarning)
0it [00:00, ?it/s]

Now pulling adsorption documents...


56533it [00:33, 1710.22it/s]


In [2]:
adsorbates = {'CO', 'H', 'OH'}

# Save as `ase.db`
For CFGP/MMS

In [7]:
from tqdm.notebook import tqdm
import ase.db
from gaspy.mongo import make_atoms_from_doc


dbs = {ads: ase.db.connect(ads + '/%s.db' % ads) for ads in adsorbates}

for doc in tqdm(docs):
    ads = doc['adsorbate']
    if ads in adsorbates:
        atoms = make_atoms_from_doc(doc)
        energy = doc['energy']
        dbs[ads].write(atoms, adsorption_energy=energy)

# Save as `doc`
For TPOT heuristic

In [12]:
import json
import copy


for ads in adsorbates:
    _docs = [copy.deepcopy(doc) for doc in docs if doc['adsorbate'] == ads]
    for doc in _docs:
        del doc['atoms']
        del doc['calc']
        del doc['results']
        del doc['mongo_id']
    with open(ads + '/%s.json' % ads, 'w') as file_handle:
        json.dump(_docs, file_handle)